In [108]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import geometry.internal_orientation as io
import warnings

warnings.filterwarnings("ignore")

# Orthorectification tests

The purpose of this notebook is to build and test the functions necessary to orthorectify KH5-ARGON images. It is divided in two sections: internal orientation (IO), for the link between fiducial and image coordinate systems, and external orientation (EO), for the link between geocentric caresian, local cartesian and WGS84 coordinate systems, and the retrieval of satellite location and attitude.

# Internal Orientation (IO)

In [111]:
x, y = 1, 12
xc, yc = 0.33, 25
alpha_t = 0 * np.pi/180
delta_xi, delta_eta = 0.1, 22.3
xi, eta = io.image_to_fiducial_coordinates(x, y, xc, yc, alpha_t, delta_xi, delta_eta)
x1, y1 = io.fiducial_to_image_coordinates(xi, eta, xc, yc, alpha_t, delta_xi, delta_eta)
x1, x, y1, y

(1.0, 1, 12.0, 12)

In [116]:
xc_t, yc_t = 1, 1
alpha_t = 30 * np.pi/180
delta_xi_t, delta_eta_t = 1, 1

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates(XI[0], XI[1], xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)) for XI in FMs_fiducial_coords]
)

res = opt.least_squares(
    io.objective_function,
    x0=[0, 0, 0, 1, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
)
params = res.x

xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates(x, y, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
xi, eta = io.image_to_fiducial_coordinates(x, y, xc, yc, alpha, delta_xi, delta_eta)
xi - xi_t, eta - eta_t

(0.08936108044652447, 0.08929586864952085)

In [117]:
xc_t, yc_t = 1, 1
alpha_t = 30 * np.pi/180
delta_xi_t, delta_eta_t = 1, 1

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array(
    [np.array(io.fiducial_to_image_coordinates(XI[0], XI[1], xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)) for XI in FMs_fiducial_coords]
)

res = opt.minimize(
    io.objective_function,
    x0=[0, 0, 0, 1, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method='BFGS',
)
if res.success == False:
    print(res)
params = res.x

xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
x, y = 1, 1
xi_t, eta_t = io.image_to_fiducial_coordinates(x, y, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
xi, eta = io.image_to_fiducial_coordinates(x, y, xc, yc, alpha, delta_xi, delta_eta)
xi - xi_t, eta - eta_t

(1.90824833389579e-07, 5.521280728280169e-08)

In [ ]:
# TODO: add fuction to transform the entire image?

# External orientation (EO)

In [ ]:
# TODO: implement WGS84 → geocentric cartesian → local cartesian link
# TODO: implement space rejection